# A script to batch render colmap with custom .PLY in Blender 2.8

## To run this script please use the blender photogrametry addon from:

  https://github.com/stuarta0/blender-photogrammetry
  
  A really great addon that imports the full colmap sparse construct scene, this means a scene that 
  constructs the cameras positions around a pointclound repressenting the scanned object.

## This script takes you from a sparse constructed colmap scene to rendered and paired images ready for traininng.

    

In [18]:

import bpy
import os
import sys
import glob
import time
from PIL import Image, ImageFile
from shutil import copyfile

Path = './'


## Loading the scene

### load_colmap(path, keep_points=False)
    A function to load a colmap scene through the addon from the path arg.
    The keep_points arg gives you the option to automatcaly delete the point cloud.

In [4]:
def load_colmap(path, keep_points=False):
    bpy.context.scene.photogrammetry.input = 'in_colmap'
    bpy.context.scene.photogrammetry.output = 'out_blender'
    bpy.context.scene.photogrammetry.in_colmap.dirpath = path
    bpy.context.scene.photogrammetry.out_blender.relative_paths = True
    bpy.context.scene.photogrammetry.out_blender.update_render_size = True
    bpy.context.scene.photogrammetry.out_blender.camera_alpha = 0
    bpy.ops.photogrammetry.process()
    if keep_points == False:
        
        bpy.ops.object.select_all(action='DESELECT')
        bpy.data.objects['PhotogrammetryPoints'].select_set(True)
        bpy.ops.object.delete() 
        
#load_colmap( "/home/gimms/Desktop/Image_2_Mesh/Rocks/rock1/sparse/0/")

### load_mesh(path, addMat = True, wireframe=False)
    A function that loads a .ply file into the scene from the path arg,
    addMat defaults True and adds a shader that shows the .ply's vertex colours,
    wireframe defaults False and is an option to create a duplicate geometry but in wireframe format

In [5]:
def load_mesh(path, addMat = True, wireframe=False):
    name = os.path.basename(os.path.normpath(path))
    bpy.ops.import_mesh.ply(filepath=path+"/"+name+"_Mesh+Col_HD.ply")
    
    ob = bpy.data.objects[name+'_Mesh+Col_HD']

    material = bpy.data.materials.get("Material")
    
    # Assign it to object
    if ob.data.materials:
        # assign to 1st material slot
        ob.data.materials[0] = material
    else:
        # no slots
        ob.data.materials.append(material)
    
    if wireframe:
        bpy.ops.import_mesh.ply(filepath=path+"/"+name+"_Mesh+Col_HD.ply")
        wireframe = bpy.data.objects[name+'_Mesh+Col_HD.001']
        frame = wireframe.modifiers.new(type = 'WIREFRAME', name = 'wireframe')
        frame.thickness = 0.0005
        bpy.ops.object.modifier_apply(apply_as='DATA', modifier=frame.name)
        # Assign it to object
        if wireframe.data.materials:
            # assign to 1st material slot
            wireframe.data.materials[0] = bpy.data.materials.get("Wireframe")
        else:
            # no slots
            wireframe.data.materials.append(bpy.data.materials.get("Wireframe"))

### LoadScene(path)
    A function that checks if the flies exist and if they do loads them and returns True, 
    if they don't returns False

In [6]:
def loadScene(path):
    name = os.path.basename(os.path.normpath(path))
    if os.path.exists(path+"/sparse/0") and os.path.exists(path+"/"+name+"_Mesh+Col_HD.ply"):
        
        load_colmap(path+"/sparse/0")  
        load_mesh(path, wireframe=False)
        
        return True
    else:
        return False


## Rendering Functions

### render_all_cam(path, size, backgroundCol)
    
    A function that runs through all the cameras in the scene and renders them out with numerical names
    to the path arg, with width/height of size and with background.
    

In [8]:
def render_all_cam(path, size, backgroundCol):
    
    bpy.data.worlds["World"].node_tree.nodes["Background"].inputs[0].default_value = backgroundCol
    scene = bpy.context.scene
    #scene.render.image_settings.file_format = 'OpenEXR' # set output format to .png
    
    bpy.context.scene.render.resolution_x = size
    bpy.context.scene.render.resolution_y = size

    
    #num= len(glob.glob("./IMagedataBase/Images_Exr/exr/*.exr"))
    num=0
    for ob in bpy.context.scene.objects:
            if ob.type == 'CAMERA':
                bpy.context.scene.camera = ob
                
                #setuv_from_camera(ob)
                # set output path so render won't get overwritten
                num+=1
                number = str(num).zfill(5)
                scene.render.filepath = path + number
                bpy.ops.render.render(write_still=True) # render still
   

### render_all_cam_encoder(path)

    A failed attempt to encode vertices, faces, normals and wertex colors to a jpeg format,
    to easilly enter it into pix2pix, but I changed course to exr encoding as I realised the
    dificulty of making a nueral net for such a complex a translation as mesh to pix was 
    probably beyond me at this point.

In [1]:
'''           
def render_all_cam_encoder(path):   
    form = []
    for ob in bpy.context.scene.objects:
        if ob.type == 'MESH':
            
            

            color_layer = ob.data.vertex_colors
            pos_layer = ob.data.vertices
            uv_layer = ob.data.uv_layers.values()
            for vert, col in zip(ob.data.vertices,color_layer):
                form.append(vert.co)
                form.append(col)
            
            #for i in range(0, len(pos_layer)-1):
                #form.append(color_layer[i])
              
                #form.append(uv_layer)


            print(form)
            
            
            
            verts_local = [v.co for v in ob.data.vertices.values()]
            verts_world = [ob.matrix_world @ v_local for v_local in verts_local]
            
            form.extend(verts_world)
            form.append((0,0,0))
            form.append((0,0,0))
            form.append((0,0,0))
            for vcol in ob.data.vertex_colors.values():
                for col in vcol.data:
                    form.append(col)
            form.append((0,0,0))
            form.append((0,0,0))
    #print(form)
    
    for ob in bpy.context.scene.objects:
        
            if ob.type == 'CAMERA':
                
                form_n_cam = form
                
                pixels = img.load() # create the pixel map

                for i in range(img.size[0]): # for every pixel:
                    for j in range(img.size[1]):
                        if pixels[i,j] != (255, 0, 0):
                            # change to black if not red
                            pixels[i,j] = (0, 0 ,0)  
'''



"           \ndef render_all_cam_encoder(path):   \n    form = []\n    for ob in bpy.context.scene.objects:\n        if ob.type == 'MESH':\n            \n            \n\n            color_layer = ob.data.vertex_colors\n            pos_layer = ob.data.vertices\n            uv_layer = ob.data.uv_layers.values()\n            for vert, col in zip(ob.data.vertices,color_layer):\n                form.append(vert.co)\n                form.append(col)\n            \n            #for i in range(0, len(pos_layer)-1):\n                #form.append(color_layer[i])\n              \n                #form.append(uv_layer)\n\n\n            print(form)\n            \n            \n            \n            verts_local = [v.co for v in ob.data.vertices.values()]\n            verts_world = [ob.matrix_world @ v_local for v_local in verts_local]\n            \n            form.extend(verts_world)\n            form.append((0,0,0))\n            form.append((0,0,0))\n            form.append((0,0,0))\n        

### clear_scene()
    
    Generic scene clearing function

In [10]:
def clear_scene():
    bpy.ops.object.select_all(action='SELECT')
    bpy.ops.object.delete() 
    

### Create Material
    creates a basic vertex color material in blender node editor

In [11]:


material = bpy.data.materials.get("Material")

if material is None:
    # create material
    material = bpy.data.materials.new(name="Material")
    material.use_nodes = True

    Diffuse_output = material.node_tree.nodes.get('Principled BSDF')
    VertexCol = material.node_tree.nodes.new('ShaderNodeVertexColor')


    # link emission shader to material
    material.node_tree.links.new(Diffuse_output.inputs[0], VertexCol.outputs[0])
    

## Print functions
    
    the colmap import prints out a lot of data so I added these in so it was easier to track the progress.
    jupyter notebook also has a non default sys.stdout so I saved it for refrence.

In [ ]:

sys_out = sys.stdout
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

# Restore
def enablePrint():
    sys.stdout = sys_out


## Main Render loop

    runing through the the folders in the Rocks directory, checking if all the components exist 
    in the directory and then processing it, outputting the renders in a new sub directory called
    'renderImages_Exr'
    
    

In [20]:

files = glob.glob(Path+'Rocks/*/')
print(len(files))
    
num=0
for rockdir in files:
    clear_scene()
    timer = time.time()
    blockPrint()
    if loadScene(rockdir):
        enablePrint()    
        render_all_cam(rockdir+"renderImages_Exr/", 512, (0,0,0, 0))
        num+=1
    else:
        enablePrint()
        print('fail')
    print("Time in Mins:")
    print((time.time()-timer)/60)
    print(str(num)+"of"+str(len(files)))
    print("done")


144
Info: Deleted 45 object(s)
Time in Mins:
0.09315592845280965
1of144
done
Info: Deleted 28 object(s)
Time in Mins:
0.04068542718887329
2of144
done
Info: Deleted 20 object(s)
Time in Mins:
0.08424564202626546
3of144
done
Info: Deleted 47 object(s)
Time in Mins:
0.09287506739298503
4of144
done
Info: Deleted 30 object(s)
Time in Mins:
0.10574572483698527
5of144
done
Info: Deleted 41 object(s)
Time in Mins:
0.020409123102823893
6of144
done
Info: Deleted 12 object(s)
Time in Mins:
0.030148224035898844
7of144
done
Info: Deleted 19 object(s)
Time in Mins:
0.08619121313095093
8of144
done
Info: Deleted 39 object(s)
Time in Mins:
0.12053001324335734
9of144
done
Info: Deleted 54 object(s)
Time in Mins:
0.20830729007720947
10of144
done
Info: Deleted 41 object(s)
Time in Mins:
0.11582736968994141
11of144
done
Info: Deleted 42 object(s)
Time in Mins:
0.13976333141326905
12of144
done
Info: Deleted 60 object(s)
Time in Mins:
0.2775170842806498
13of144
done
Info: Deleted 61 object(s)
Time in Mins:
0

Time in Mins:
0.10819861888885499
109of144
done
Info: Deleted 26 object(s)
Time in Mins:
0.07214953104654948
110of144
done
Info: Deleted 32 object(s)
Time in Mins:
0.05786710977554321
111of144
done
Info: Deleted 26 object(s)
Time in Mins:
0.11993951797485351
112of144
done
Info: Deleted 41 object(s)
Time in Mins:
0.08996933301289876
113of144
done
Info: Deleted 41 object(s)
Time in Mins:
0.09415579239527384
114of144
done
Info: Deleted 33 object(s)
Time in Mins:
0.15292954842249554
115of144
done
Info: Deleted 39 object(s)
Time in Mins:
0.1585541566212972
116of144
done
Info: Deleted 50 object(s)
Time in Mins:
0.04836207628250122
117of144
done
Info: Deleted 18 object(s)
Time in Mins:
0.08182316621144613
118of144
done
Info: Deleted 31 object(s)
Time in Mins:
0.21556594371795654
119of144
done
Info: Deleted 64 object(s)
Time in Mins:
0.026487298806508384
120of144
done
Info: Deleted 14 object(s)
Time in Mins:
0.07192110220591227
121of144
done
Info: Deleted 27 object(s)
Time in Mins:
0.021591405

## Merging the dataSet for EXR

    A set of functions that copy and paste all of the rendered .exr and original matching .jpg into one folder.
    Numbering them in pairs, this is not as secure as having the two sides of the set merged but it worked for 
    in the time scale.

In [ ]:


def load_images_from_folder(folder):
    images = sorted(os.listdir(folder))
   
    return images


In [ ]:

def copy_images(path, num):
    
   # print(path)
    if os.path.exists(path+"images") and  os.path.exists(path+"renderImages_Exr") :
        print(path)
        
        images_base=load_images_from_folder(path+'images/')
        images_render=load_images_from_folder(path+'renderImages_Exr/')
        print(len(images_base))
        
        if len(images_base)==len(images_render):
            for n  in range(0,len(images_base)):
                num+=1
                jpg = images_base[n]
                copyfile(path+'images/'+jpg, os.getcwd()+'/imgDatabase/rockPair-exr/'+str(num).zfill(5)+'.jpg')
                exr = images_render[n]
                copyfile(path+'renderImages_Exr/'+exr, os.getcwd()+'/imgDatabase/rockPair-exr/'+str(num).zfill(5)+'.exr')
        print(len(images_render))    
        return num
        #else:
         #   return num
    else:
        print('fail')
        return num
    

In [ ]:

allRocks = glob.glob(Path+'/Rocks/*/')

num= len(glob.glob(os.getcwd()+'/imgDatabase/rockPair-exr/*.jpg'))
for rockdir in allRocks:
    timer = time.time()
    num = copy_images(rockdir, num)
    print("Time in Mins:")
    print((time.time()-timer)/60)
    #print(str(num)+"of"+str(totFiles))
    print("done")
print("Actually Done!!!!")

## Merge Dataset for .jpg

    For the original pix2pix setup, this checks the image set bothe exist and then merges them into one solid 
    .jpg ready for training, saving all the merged into a single folder ready for splitting into train and
    test.

In [ ]:
def load_images_from_folder(folder):
    images = []
    for filename in sorted(os.listdir(folder)):
        img = Image.open(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

In [ ]:

def merge_imagesDataset(path, num):
    
   # print(path)
    if os.path.exists(path+"images") and  os.path.exists(path+"renderImages") :
      
        
        images_base=load_images_from_folder(path+'images/')
        images_render=load_images_from_folder(path+'renderImages/')
        
        if len(images_base)==len(images_render):
            for n  in range(0,len(images_base)):
                images_2_merge = [images_base[n],images_render[n]]
                widths, heights = zip(*(i.size for i in images_2_merge))
                total_width = sum(widths)
                max_height = max(heights)
                new_im = Image.new('RGB', (total_width, max_height))
                new_im.paste(images_render[n],(widths[0],0))
                new_im.paste(images_base[n], (0,0))


                num+=1
                new_im.save(os.getcwd()+'/imgDatabase/rockPair_wireframe/'+str(num).zfill(5)+'.jpg')
        print(len(images_render))    
        return num
        #else:
         #   return num
    else:
        print('fail')
        return num
    

In [ ]:

allRocks = glob.glob(Path+'/Rocks/*/')
print(len(allRocks))
num= 2668
for rockdir in allRocks:
    timer = time.time()
    num = merge_images(rockdir, num)
    print("Time in Mins:")
    print((time.time()-timer)/60)
    #print(str(num)+"of"+str(totFiles))
    print("done")
print("Actually Done!!!!")